In [77]:
%matplotlib inline
import imp
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn

In [78]:
from problem import get_train_data

X_df, y_array = get_train_data()

In [79]:
X_df.head(5)

,Legal_ID,Name,Activity_code (APE),Address,Zipcode,City,Headcount,Fiscal_year_end_date,Fiscal_year_duration_in_months,Year
0,582296,COMICOB,NaN,BOULOUPARIS BP 15 BOULOUPARIS,NaN,NaN,NaN,2016-12-31,12.0,2016.0
1,582981,CIANFARANI JEAN-MICHEL,NaN,33 AVENUE H. LAFLEUR - VICTOIRE - B.P. 4031 NO...,NaN,NaN,NaN,2016-12-31,12.0,2016.0
2,588541,OK POULET 5EME SARL,NaN,23 RUE PAUL BERT,98800.0,NOUMEA,1.0,2016-12-31,12.0,2016.0
3,589275,INSTITUT DE LA QUALITE S.A.R.L.,NaN,224 RUE J. IEKAWE - COMPLEXE DE LA BELLE-VIE -...,98800.0,NOUMEA,NaN,2016-12-31,12.0,2016.0
4,592972,EXPERT TRADE,NaN,2 RUE DE L ASTROLABE OUEMO MAGENTA,98800.0,NOUMEA,NaN,2016-12-31,12.0,2016.0


In [80]:
X_df.shape

(1495948, 10)

## Award notices

In [81]:
award = pd.read_csv('data/award_notices_RAMP.csv.zip', compression='zip')
award.head(5)

/Users/QQINO/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CallID,Publication_date,End_of_call_date,Departments_of_publication,Departments_of_provision,Call_summary,Call_title,Complete_call_description,Total_amount,CPV_classes,...,ID,awarded,description,incumbent_name,incumbent_address,incumbent_zipcode,incumbent_city,incumbent_country,number_of_received_bids,amount
0,16-119770,2016-08-11 00:00:00,2016-09-20,13,NaN,mission de maitrise d'oeuvre relative aux trav...,MISSION DE MAITRISE D'OEUVRE RELATIVE AUX TRAV...,MISSION DE MAITRISE D'OEUVRE RELATIVE AUX TRAV...,NaN,71000000,...,NaN,Yes,NaN,AI PROJECT,"11, avenue de la Capelette",13010,Marseille,FR,13.0,83200.0
1,15-46335,2015-03-27 00:00:00,2015-04-25,85,85,fourniture de réactifs immunosérologiques pour...,NaN,Fourniture de réactifs immunosérologiques pour...,NaN,33696200 33696500,...,1,Yes,Kit pour recherche d'anticorps anti-IBR sur sé...,ID-VET,310 rue Louis Pasteur,34790,Grabels,FR,2.0,NaN
2,15-46335,2015-03-27 00:00:00,2015-04-25,85,85,fourniture de réactifs immunosérologiques pour...,NaN,Fourniture de réactifs immunosérologiques pour...,NaN,33696200 33696500,...,2,Yes,Kit pour recherche d'anticorps anti-IBR sur mé...,IDEXX MONTPELLIER SAS,323 RUE DE LA GALERA,34090,MONTPELLIER,FR,2.0,NaN
3,15-46335,2015-03-27 00:00:00,2015-04-25,85,85,fourniture de réactifs immunosérologiques pour...,NaN,Fourniture de réactifs immunosérologiques pour...,NaN,33696200 33696500,...,3,Yes,Kit pour recherche d'anticorps anti-IBR sur sé...,IDEXX MONTPELLIER SAS,323 RUE DE LA GALERA,34090,MONTPELLIER,FR,2.0,NaN
4,15-46335,2015-03-27 00:00:00,2015-04-25,85,85,fourniture de réactifs immunosérologiques pour...,NaN,Fourniture de réactifs immunosérologiques pour...,NaN,33696200 33696500,...,4,Yes,Kit pour recherche d'anticorps gE contre l'IBR...,ID-VET,310 rue Louis Pasteur,34790,Grabels,FR,2.0,NaN


In [82]:
# list of numerical columns
num_cols = ['Legal_ID', 'Headcount']

In [83]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean'))
])

In [84]:
from sklearn.preprocessing import FunctionTransformer

def process_date(X):
    date = pd.to_datetime(X['Fiscal_year_end_date'], format='%Y-%m-%d')
    return np.c_[date.dt.year, date.dt.month, date.dt.day]

date_transformer = FunctionTransformer(process_date, validate=False)

In [85]:
from sklearn.preprocessing import FunctionTransformer

def process_date_ordinal(X):
    date = pd.to_datetime(X['Fiscal_year_end_date'], format='%Y-%m-%d')
    ord_date = date.apply(lambda x: x.toordinal()).values[:,np.newaxis]
    return ord_date - pd.Timestamp('2013-01-01').toordinal()

date_ord_transformer = FunctionTransformer(process_date_ordinal, validate=False)

In [86]:
def process_numeric_APE(X):
    APE_prefix = X['Activity_code (APE)'].str[:2]
    APE_root = X['Activity_code (APE)'].str[2]
    val_prefix = pd.to_numeric(APE_prefix, errors='coerce').values[:, np.newaxis]
    val_root = pd.to_numeric(APE_root, errors='coerce').values[:, np.newaxis]
    return np.concatenate((val_prefix,val_root),axis=1)

APE_n_transformer = FunctionTransformer(process_numeric_APE, validate=False); 

In [87]:
def process_string_APE(X):
    return X['Activity_code (APE)'].str[3:].values[:, np.newaxis]

APE_s_transformer = FunctionTransformer(process_string_APE, validate=False);

In [88]:
def zipcodes(X):
    zipcode_nums = pd.to_numeric(X['Zipcode'], downcast='integer', errors='coerce')
    val_prefix = zipcode_nums.values[:, np.newaxis]/1000
    val_suffix = zipcode_nums.values[:, np.newaxis]%1000
    return np.concatenate((val_prefix,val_suffix),axis=1)

zipcode_transformer = FunctionTransformer(zipcodes, validate=False)

In [192]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline

zipcode_col = ['Zipcode']
date_cols = ['Fiscal_year_end_date']
drop_cols = ['Name', 'Address', 'City','Fiscal_year_duration_in_months','Year']
APE_col = ['Activity_code (APE)']
num_cols = ['Legal_ID', 'Headcount']


preprocessor = ColumnTransformer(
    transformers=[
        ('zipcode', make_pipeline(zipcode_transformer, SimpleImputer(strategy='constant', fill_value=0)), zipcode_col),
        ('num', numeric_transformer, num_cols),
        ('date', make_pipeline(date_ord_transformer, SimpleImputer(strategy='mean')), date_cols),
        ('APE_n', make_pipeline(APE_n_transformer, SimpleImputer(strategy='constant', fill_value=0)), APE_col),
        #('APE_s', make_pipeline(APE_s_transformer, SimpleImputer(strategy='constant', fill_value='0'), OrdinalEncoder()), APE_col),
        ('drop cols', 'drop', drop_cols),
    ])


In [193]:
# make sure it works
a = preprocessor.fit_transform(X_df)

In [194]:
a.shape

(1495948, 7)

In [195]:
a

array([[0.00000000e+00, 0.00000000e+00, 5.82296000e+05, ...,
        1.46000000e+03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 5.82981000e+05, ...,
        1.46000000e+03, 0.00000000e+00, 0.00000000e+00],
       [9.88000000e+01, 8.00000000e+02, 5.88541000e+05, ...,
        1.46000000e+03, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 9.95750288e+08, ...,
        3.95000000e+02, 2.80000000e+01, 2.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 9.95750288e+08, ...,
        7.29000000e+02, 2.80000000e+01, 2.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 9.95750288e+08, ...,
        1.09400000e+03, 2.80000000e+01, 2.00000000e+00]])

In [196]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=10)

In [206]:
from sklearn.metrics import make_scorer

def loss(y_true, y_pred):
    
    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    true = np.maximum(5., np.log10(np.maximum(1., y_true)))
    pred = np.maximum(5., np.log10(np.maximum(1., y_pred)))
    
    loss = np.mean(np.abs(true - pred))
    
    return loss
    
fan_loss = make_scorer(loss, greater_is_better=False)

In [207]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit

clf = Pipeline(steps=[
    ('preprocessing', preprocessor_comp),
    ('classifier', regressor)])

cv = GroupShuffleSplit(n_splits=8, test_size=0.25)

scores_Xdf = -cross_val_score(clf, X_df, y_array, cv=cv, scoring=fan_loss, groups=X_df['Legal_ID'], n_jobs=3)

print("mean: %e (+/- %e)" % (scores_Xdf.mean(), scores_Xdf.std()))

mean: 4.248152e-01 (+/- 1.176472e-03)


## Naive merge

In [199]:
award['Name_processed'] = award['incumbent_name'].str.lower()
award['Name_processed'] = award['Name_processed'].str.replace('[^\w]','')

In [200]:
award_features = award.groupby(['Name_processed'])['amount'].agg(['count','sum'])
award_features.head(3)

,count,sum
Name_processed,,
,1,55479.36
0,1,0.00
00000,0,0.00


In [201]:
def merge_naive(X):
    X['Name'] = X['Name'].str.lower()     
    X['Name'] = X['Name'].str.replace('[^\w]','')
    df = pd.merge(X, award_features, left_on='Name', right_on='Name_processed', how='left')
    return df[['count','sum']]
merge_transformer = FunctionTransformer(merge_naive, validate=False)

In [202]:
zipcode_col = ['Zipcode']
date_cols = ['Fiscal_year_end_date']
drop_cols = ['Name', 'Address', 'City','Fiscal_year_duration_in_months','Year']
APE_col = ['Activity_code (APE)']
merge_col = ['Name']

preprocessor = ColumnTransformer(
    transformers=[
        ('zipcode', make_pipeline(zipcode_transformer, SimpleImputer(strategy='constant', fill_value=0)), zipcode_col),
        ('num', numeric_transformer, num_cols),
        ('date', make_pipeline(date_ord_transformer, SimpleImputer(strategy='mean')), date_cols),
        ('APE_n', make_pipeline(APE_n_transformer, SimpleImputer(strategy='constant', fill_value=0)), APE_col),
        ('merge', make_pipeline(merge_transformer, SimpleImputer(strategy='median')), merge_col),
        ('drop cols', 'drop', drop_cols),
    ])

In [203]:
# check it works
preprocessor.fit_transform(X_df)

array([[0.00000000e+00, 0.00000000e+00, 5.82296000e+05, ...,
        0.00000000e+00, 1.00000000e+00, 2.15600900e+05],
       [0.00000000e+00, 0.00000000e+00, 5.82981000e+05, ...,
        0.00000000e+00, 1.00000000e+00, 2.15600900e+05],
       [9.88000000e+01, 8.00000000e+02, 5.88541000e+05, ...,
        0.00000000e+00, 1.00000000e+00, 2.15600900e+05],
       ...,
       [0.00000000e+00, 0.00000000e+00, 9.95750288e+08, ...,
        2.00000000e+00, 9.00000000e+00, 3.19457147e+06],
       [0.00000000e+00, 0.00000000e+00, 9.95750288e+08, ...,
        2.00000000e+00, 9.00000000e+00, 3.19457147e+06],
       [0.00000000e+00, 0.00000000e+00, 9.95750288e+08, ...,
        2.00000000e+00, 9.00000000e+00, 3.19457147e+06]])

In [204]:
regressor = RandomForestRegressor(n_estimators=10)

In [205]:
clf = Pipeline(steps=[
    ('preprocessing', preprocessor_merge),
    ('classifier', regressor)])

cv = GroupShuffleSplit(n_splits=8, test_size=0.25)

scores_merge = -cross_val_score(clf, X_df, y_array, cv=cv, scoring=fan_loss, groups=X_df['Legal_ID'],
                               n_jobs=3)

print("mean: %e (+/- %e)" % (scores_merge.mean(), scores_merge.std()))

mean: 4.191072e-01 (+/- 1.033031e-03)


In [ ]:
!ramp_test_submission --submission my_model_1

Testing Prediction of annual revenue using FAN
Reading train and test files from ./data ...
Reading cv ...
Training submissions/my_model_1 ...
CV fold 0
	score  fan error        time
	train       0.12  154.507412
	valid       0.42   23.254556
	test        0.42    7.499385
CV fold 1
	score  fan error        time
	train       0.11  142.870552
	valid       0.42   25.926292
	test        0.42    7.699212
CV fold 2
	score  fan error        time
	train       0.13  129.905554
	valid       0.42   18.666847
	test        0.42    6.297429
CV fold 3
	score  fan error        time
	train       0.12  136.055322
	valid       0.42   21.226836
	test        0.42    8.588347
CV fold 4
